# Initialisations

In [1]:
import requests, json
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import sys
import time
import requests, json
import numpy as np
import calendar
from datetime import datetime
import random
import glob
import hashlib

In [2]:
def tounixtime(datetime_input):
    return(calendar.timegm(datetime_input.utctimetuple()))

def todatetimeformat(utctime):
    year=int(utctime[0])*1000 + int(utctime[1])*100 + int(utctime[2])*10 + int(utctime[3])
    month=int(utctime[5])*10 + int(utctime[6])
    day=int(utctime[8])*10 + int(utctime[9])
    hr=int(utctime[11])*10 + int(utctime[12])
    minute=int(utctime[14])*10 + int(utctime[15])
    second=int(utctime[17])*10 + int(utctime[18])
    millisecond=int(utctime[20])*100 + int(utctime[21])*10 + int(utctime[22])   
    return(datetime(year,month,day,hr,minute,second,millisecond))

def toutcformat(datetime_input):
    tstr=str(datetime_input)
    year=tstr[0]+tstr[1]+tstr[2]+tstr[3]
    month=tstr[5]+tstr[6]
    day=tstr[8]+tstr[9]

    try:
        if type(int(tstr[11]+tstr[12]))==int:
            hour=str(tstr[11]+tstr[12])
    except:
        hour='00'             #no hours given       

    try:
        if type(int(tstr[14]+tstr[15]))==int:
            minute=str(tstr[14]+tstr[15])
    except:
        minute='00'             #no minutes given

    try:
        if type(int(tstr[17]+tstr[18]))==int:
            second=str(tstr[17]+tstr[18])
    except:
        second='00'             #no seconds given

    try:
        if type(int(tstr[20]+tstr[21]+tstr[22]))==int:
            millisecond=str(tstr[20]+tstr[21]+tstr[22])
    except:
        millisecond='000'       #no milliseconds given



    utctime=year + '-' + month + '-' + day + 'T' + hour + ':' + minute + ':' + second + '.' + millisecond + 'Z'
    return utctime


def addminutesutc(utctime,mins):
    return(toutcformat(datetime.utcfromtimestamp(tounixtime(todatetimeformat(utctime))+(mins*60))))

In [3]:
#converts numpy types to python types, otherwise json conversion produces an error. call json.dumps(***, cls=MyEncoder)
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

        

# returns strings in the conventional format for iot.ids
def idstr(idinput):
    return(str(idinput).lower().replace(" ", "_").replace("/", "_").replace("ä", "ae").replace("ö", "oe").replace("ü", "ue").replace("ß", "ss"))


# returns the first 7 digits of the sha1 hash of the input string
def hashfunc(inputstring, printme):
    returnhash= hashlib.sha1(bytes(str(inputstring), 'utf-8')).hexdigest()[0:7]
    if printme == True:
        print("Converting '" + str(inputstring) + "' to hash '" + str(returnhash) +"'")
    return(returnhash)



# returns the full 40 digits of the sha1 hash of the input string
def hashfuncfull(inputstring, printme):
    returnhash= hashlib.sha1(bytes(str(inputstring), 'utf-8')).hexdigest()
    if printme == True:
        print("Converting '" + str(inputstring) + "' to hash '" + str(returnhash) +"'")
    return(returnhash)



#function that takes a link and gives an array with the link itself and all nextlinks
def chainnextlinks(link):
    if "?$" in link:
        querysign="&$"
    else:
        querysign="?$"
        
    count = json.loads(requests.get(link + querysign + "count=True").text)["@iot.count"] #how many entites in the link
    chunks = 1000 #how many you want to fit on one page
    baselink = str(link + querysign + 'top=' + str(chunks))
    
    i=1
    links=[baselink]
    while count > chunks:
        links.append(baselink + '&$skip=' + str(i*chunks))
        count = count-chunks
        i+=1
    return(links)

In [4]:
baseurl = 'https://www.umweltbundesamt.de/js/uaq/data/stations' #get data where from
url = "http://smartaqnet.teco.edu/v1.0" #post data where to

scope='1SMW' #umweltbundesamt website code: 1 hour means
scopesec= 60*60 #scope in seconds, needed for interval to tag the next observation
feature='PM10'

obsproperty_code = "mcpm10"

# Program

In [29]:
# get all existing stations from database
# grab value and post to server




#for eachlink in chainnextlinks(url + "/Things?$filter=properties/operator_url%20eq%20%27lfu.bayern.de%27"):


#alle things von lfu.bayern.de
listofthings=json.loads(requests.get(url + "/Things?$filter=properties/operator_url%20eq%20%27lfu.bayern.de%27").text)["value"]


for thing in listofthings:
    
    try:
        pm10streamid=json.loads(requests.get(url + "/Things('" + thing["@iot.id"] + "')/Datastreams?$filter=ObservedProperty/@iot.id%20eq%20%27saqn:op:" + obsproperty_code + "%27&$select=@iot.id").text)["value"][0]["@iot.id"]
    except:
        print("no datastream existing for thing " + thing["name"] + " with observedProperty " + obsproperty_code)
        continue
    
    datastream=json.loads(requests.get("http://smartaqnet.teco.edu/v1.0/Datastreams('" + pm10streamid + "')").text)

    try:
        end_latest_pheno_time = json.loads(requests.get("http://smartaqnet.teco.edu/v1.0/Datastreams('" + pm10streamid + "')/Observations?$orderby=phenomenontime%20desc&$top=1").text)["value"][0]["phenomenonTime"][-24:]
    except: 
        print("no data available on datastream " + pm10streamid)
        continue
    
    if tounixtime(datetime.utcnow())-tounixtime(todatetimeformat(end_latest_pheno_time)) > scopesec + 10:
        begintimeunix=tounixtime(todatetimeformat(end_latest_pheno_time))
        endtimeunix=tounixtime(todatetimeformat(end_latest_pheno_time)) + scopesec

        #get data from uba
        getdatafrom=baseurl + '/measuring?pollutant[]=' + feature + '&scope[]=' + scope + '&station[]=' + thing["properties"]["station_code"] + '&group[]=pollutant&range[]=' + str(begintimeunix+10) + ',' + str(endtimeunix+10)
        datavalue=json.loads(requests.get(getdatafrom).text)["data"][0][0][0]





        #get the unhashed datastream id
        #thing id           
        thing_id_url = thing["properties"]["operator_url"]
        thing_id_thingname = str(thing["name"])
        thing_id_date = str(thing["properties"]["station_start_date"])[0:4] + "-" + str(thing["properties"]["station_start_date"])[4:6]
        thing_id_thingnumber = str(thing["properties"]["station_code"])

        thing_tohash = idstr(thing_id_url + ":" + thing_id_thingname + ":" + thing_id_date + ":" + thing_id_thingnumber)

        #Datastream ID
        stream_id_url = datastream["properties"]["operator_url"] #sollte eigentlich auch in den properties des datastreams stehen
        stream_id_sensorname = datastream["properties"]["sensor_name"]
        stream_id_sensornumber = datastream["properties"]["sensor_serial_number"]

        stream_tohash = idstr(stream_id_url + ":" + stream_id_sensorname + ":" + stream_id_sensornumber)

        #ObservedProperty


        fullstream_tohash = thing_tohash + ":" + stream_tohash + ":" + obsproperty_code

        #possible check function if idgeneration was correct
#        if "saqn:ds:" + hashfunc(fullstream_tohash,False) == pm10streamid:
#            print("true")
#        else:
#            print("stream id not correct: ")
#            print("    " + pm10streamid + " != " + "saqn:ds:" + hashfunc(fullstream_tohash,False))
#            print("    " + "tried to hash " + fullstream_tohash)




        #generate observation and push to frost
        observation_id_prefix = "saqn:o:"
        observation_interval = end_latest_pheno_time + "/" + "PT1H"

        observation_tohash = idstr(fullstream_tohash + ":" + observation_interval) #!!!! fullstream nicht da

        generateobsid = observation_id_prefix + hashfuncfull(observation_tohash, False)

        observation = {
        "phenomenonTime" : observation_interval, 
        "result" : datavalue,
        "Datastream":{"@iot.id": str(pm10streamid)},
        "@iot.id": generateobsid
        }
        requests.post(url + '/Observations', json.dumps(observation))



no data available on datastream saqn:ds:80af8f6
no datastream existing for thing Station Augsburg/Haunstetten with observedProperty mcpm10
